**Libraries importieren**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from IPython.display import display, HTML
import random

**Classifier**

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
spam = pd.read_csv('./spambase.csv').fillna(0)
spam.head()

bank = pd.read_csv('./banknote_authentication.csv').fillna(0)
bank.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
X_spam = spam.iloc[:,:57].values
Y_spam = spam['Target'].values

X_bank = bank.iloc[:,:4].values
Y_bank = bank['class'].values

## Bagging

Hier wurde die `bagging` Methode verwendet um einen Wert vorherzusagen

### Beschreibung

Ein Argument gegen den Einsatz von gänzlich verschiedenen Algortihmen ist, dass ein solcher Ensemble Learner nur schwer zu verstehen und einzuschätzen ist (übrigens ein generelles Problem im maschinellen Lernen). Bereits ein einzelner Algorithmus (z. B. Support Vector Machine) kann nach jedem Training alleine auf Basis der jeweils ausgewählten Daten (zum Training und zum Testen) recht unterschiedlich in seiner Vorhersage ausfallen.  
Bagging (kurze Form von Bootstrap Aggregation) ist ein Ensemble Learning Prinzip, bei dem grundsätzlich der gleiche Algorithmus parallel mit unterschiedlichen Aufteilungen der Daten trainiert (und natürlich getestet) wird. Die Aufteilung der Daten kann dabei komplett (der vollständige Datensatz wird verteilt und verwendet) oder auch nur über Stichproben erfolgen (dann gibt es mehrfach verwendete Datenpunkte, aber auch solche, die überhaupt nicht verwendet werden). Das Ziel ist dabei insbesondere, im Endergebnis Unter- und Überanpassung zu vermeiden. Gibt es viele Dichte-Cluster und Ausreißer in den Daten, wird nicht jeder Klassifizierer sich diesen angepasst haben können. Jede Instanz der Klassifizierer erhält weitgehend unterschiedliche Daten mit eigenen Ausreißern und Dichte-Clustern, dabei darf es durchaus Überschneidungen bei der Datenaufteilung geben.

### Umsetzung

#### Aufteilung der Datensätze in Train & Test

In [5]:
X_train_spam, X_test_spam, y_train_spam, y_test_spam = train_test_split(X_spam, Y_spam, test_size=0.8, random_state=0)

X_train_bank, X_test_bank, y_train_bank, y_test_bank = train_test_split(X_bank, Y_bank, test_size=0.8, random_state=0)

#### Spambase Datensatz

##### Baggin

Parameter-Setting:  
* max_sample: Es werden 50% der zur Verfügung stehenden Daten verwendet
* max_features: 100% der zur Verfügung stehenden Daten werden verwendet
* n_estimators: Es werden 20 Decision-Trees gepflanzt hihi

In [6]:
bg = BaggingClassifier(DecisionTreeClassifier(), max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(X_train_spam, y_train_spam)
bg.score(X_test_spam, y_test_spam)

0.9991850040749797

##### Random Forest

Parameter-Setting:
* n_estimators: Anzahl der gepflanzten Bäume

In [7]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(X_train_spam, y_train_spam)
rf.score(X_test_spam, y_test_spam)

0.9970116816082586

##### AdaBoost

Parameter-Setting:
* DecisionTreeClassifier: Basisklassifizierer
* n_estimators: Anzahl
* learning_rate: Lernrate

In [8]:
adb = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 5, learning_rate = 1)
adb.fit(X_train_spam, y_train_spam)
adb.score(X_test_spam, y_test_spam)

0.9997283346916599

### Hyperparemeteroptimierung mit `GridsearchCV`

In [25]:
parameters = {'n_estimators': [5, 10, 15], 
              'max_features': [1.0], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 5, 10], 
              'min_samples_split': [2, 3, 8],
              'min_samples_leaf': [1,3,8],
              'min_weight_fraction_leaf':[0, 0.1, 0.5],
              'max_leaf_nodes':[None],
              'min_impurity_decrease':[0],
              'min_impurity_split':[1e-7]
             }

acc_scorer = make_scorer(accuracy_score)

In [28]:
bg = BaggingClassifier(DecisionTreeClassifier())

rf = RandomForestClassifier()

adb = AdaBoostClassifier(DecisionTreeClassifier())

In [30]:
grid_obj = GridSearchCV(bg, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train_spam, y_train_spam)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: Invalid parameter criterion for estimator BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                        

In [ ]:
# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train_spam, y_train_spam)

In [ ]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))